In [10]:
# Import libraries
import base64
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter

# Initialize MongoDB connection
shelter = AnimalShelter()

# Retrieve initial data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Initialize the Dash app
app = JupyterDash('CS-340 Dashboard - Faris Malik')

# Layout for the dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Faris Malik'))),
    
    # Grazioso Salvare logo with link
    html.A(
        html.Img(src='assets/Grazioso Salvare Logo.png', style={'height': '100px'}),
        href='https://www.snhu.edu',
        target='_blank'
    ),
    
    html.Hr(),
    
    # Interactive Filter Options (Horizontal layout, left-aligned)
    dcc.RadioItems(
        id='rescue-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        labelStyle={'display': 'inline-block', 'marginRight': '10px'},  # Horizontal layout with spacing
        style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'flex-start'}  # Align left
    ),
    
    html.Br(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="single",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_size=10  # Limits number of rows visible per page
    ),
    
    html.Br(),
    
    # Pie Chart and Geolocation Map side by side
    html.Div([
        # Pie chart
        html.Div([
            dcc.Graph(id='pie-chart')
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        # Geolocation Map
        html.Div([
            html.Div(id='map-id', className='col s12 m6')
        ], style={'width': '48%', 'display': 'inline-block', 'paddingLeft': '20px'})
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    
    html.Br(),
    
    # Reset Button
    html.Button('Reset', id='reset-button', n_clicks=0),
    
    html.Hr(),
    
    # Unique identifier
    html.Div("Unique ID: Faris Malik", style={'marginTop': '20px'})
])

# Callback to filter data and update table
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-filter', 'value')]
)
def filter_data(rescue_type):
    if rescue_type == "Water Rescue":
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "sex_upon_outcome": {"$regex": "Intact Female"}
        }
    elif rescue_type == "Mountain Rescue":
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "sex_upon_outcome": {"$regex": "Intact Male"}
        }
    elif rescue_type == "Disaster Rescue":
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
            "sex_upon_outcome": {"$regex": "Intact Male"}
        }
    else:  # Reset
        query = {}

    # Fetch filtered data from MongoDB
    filtered_data = shelter.read(query)
    return pd.DataFrame(filtered_data).to_dict('records')

# Callback to update pie chart based on filtered data
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if not data:
        return {}
    df = pd.DataFrame(data)
    fig = px.pie(df, names='breed', title='Breed Distribution')
    return fig

# Callback to update geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Ensure that derived_virtual_data is not None or empty
    if not viewData:
        return [html.P("No data available to display on the map.")]
    
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default to the first row if no row is selected or index is invalid
    row = 0 if index is None or len(index) == 0 else index[0]

    # Default map center (Austin, TX)
    center_lat, center_lon = 30.75, -97.48
    
    # Ensure the row exists in the DataFrame and retrieve valid coordinates
    if row < len(dff):
        if 'location_lat' in dff.columns and 'location_long' in dff.columns:
            center_lat = dff.iloc[row]['location_lat']
            center_lon = dff.iloc[row]['location_long']
    
    # Ensure proper handling of animal breed and name columns
    breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown Breed"
    name = dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown Name"
    
    # Render map with marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[center_lat, center_lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[center_lat, center_lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

# Callback to reset dashboard
@app.callback(
    Output('rescue-filter', 'value'),
    [Input('reset-button', 'n_clicks')]
)
def reset_dashboard(n_clicks):
    if n_clicks > 0:
        return 'Reset'
    return dash.no_update

# Run the Dash app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10685/
